In [197]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import scipy.stats as stats

In [198]:
df = pd.read_csv('dane.csv', sep=';', index_col=0)

In [199]:
df.head(10)

,Cel,Aktywa,Płeć,Stan cywilny,Mieszkanie,Praca,Okres,Wiek,Ocena
1,Drobne AGD,Bardzo niskie,M,Samotna(y),Własne,Fizyczny,13,23,Dobra
2,Meble,Niskie,M,Rozwiedziona(y),Własne,Umysłowy,25,32,Zła
3,Nowy samochód,Bardzo niskie,M,Samotna(y),Własne,Umysłowy,19,38,Zła
4,Meble,Bardzo niskie,M,Samotna(y),Własne,Fizyczny,13,36,Zła
5,Edukacja,Niskie,M,Samotna(y),Wynajem,Umysłowy,40,31,Dobra
6,Meble,Brak,M,Stały związek,Własne,Umysłowy,11,25,Dobra
7,Nowy samochód,Bardzo niskie,M,Stały związek,Własne,Fizyczny,13,26,Dobra
8,Business,Bardzo niskie,M,Samotna(y),Własne,Fizyczny,14,27,Dobra
9,Drobne AGD,Bardzo niskie,M,Samotna(y),Własne,Umysłowy,37,25,Zła
10,Drobne AGD,Brak,K,Rozwiedziona(y),Własne,Umysłowy,25,43,Zła


In [200]:
# Kodowanie zero jedynkowo
df['Płeć'] = df['Płeć'].apply(lambda x: 1 if x == 'M' else 0)

df['Stan cywilny'] = df['Stan cywilny'].apply(lambda x: 1 if x == 'Stały związek' else 0)

df['Mieszkanie'] = df['Mieszkanie'].apply(lambda x: 1 if x == 'Własne' else 0)

df['Praca'] = df['Praca'].apply(lambda x: 1 if x == 'Umysłowy' else 0)

df['Ocena'] = df['Ocena'].apply(lambda x: 1 if x == 'Dobra' else 0)

In [201]:
df.isna().any()

Cel             False
Aktywa          False
Płeć            False
Stan cywilny    False
Mieszkanie      False
Praca           False
Okres           False
Wiek            False
Ocena           False
dtype: bool

In [202]:
df.head(10)

,Cel,Aktywa,Płeć,Stan cywilny,Mieszkanie,Praca,Okres,Wiek,Ocena
1,Drobne AGD,Bardzo niskie,1,0,1,0,13,23,1
2,Meble,Niskie,1,0,1,1,25,32,0
3,Nowy samochód,Bardzo niskie,1,0,1,1,19,38,0
4,Meble,Bardzo niskie,1,0,1,0,13,36,0
5,Edukacja,Niskie,1,0,0,1,40,31,1
6,Meble,Brak,1,1,1,1,11,25,1
7,Nowy samochód,Bardzo niskie,1,1,1,0,13,26,1
8,Business,Bardzo niskie,1,0,1,0,14,27,1
9,Drobne AGD,Bardzo niskie,1,0,1,1,37,25,0
10,Drobne AGD,Brak,0,0,1,1,25,43,0


In [203]:
df.iloc[:,2:-1].columns

Index(['Płeć', 'Stan cywilny', 'Mieszkanie', 'Praca', 'Okres', 'Wiek'], dtype='object')

In [204]:
y = df['Ocena']
y


1      1
2      0
3      0
4      0
5      1
      ..
421    1
422    0
423    0
424    0
425    1
Name: Ocena, Length: 425, dtype: int64

In [205]:
def hosmer_lemeshow(y, y_pred_prob, n_bins=10):
    y = np.array(y)
    y_pred_prob = np.array(y_pred_prob)

    # Tworzymy biny dla prognozowanych prawdopodobieństw
    percentiles = np.percentile(y_pred_prob, np.linspace(0, 100, n_bins + 1))
    bins = np.digitize(y_pred_prob, percentiles[1:-1])
    
    # Liczymy rzeczywiste i oczekiwane proporcje sukcesów
    actual = np.bincount(bins, weights=y) / np.bincount(bins)
    expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
    
    # Obliczamy statystykę testu Hosmera-Lemeshowa
    hl_stat = np.sum((actual - expected) ** 2 / (expected * (1 - expected) / np.bincount(bins)))
    
    # Obliczamy p-wartość
    p_value = stats.chi2.sf(hl_stat, n_bins - 2)
    
    return hl_stat, p_value

Testy

In [218]:
def evaluate(*args):
    evaluation = pd.DataFrame({
    'Column' : [],
    'Std dev' : [],
    'AIC' : [],
    'BIC' : [],
    'R2_Nagelkera' : [],
    'Odds ratio' : [],
    'Hosmel test': [],
    'Significance': [],
    })

    y = df['Ocena']
    
    args = list(args)

    for column in df.iloc[:,2:-1].drop(args, axis=1).columns:
        args.append(column)
        X = sm.add_constant(df[args])  # dodajemy stałą do modelu
        model = sm.Logit(y, X)
        results = model.fit()
        
        # Obliczanie p-wartości dla każdego parametru
        p_values = results.pvalues
        print(f"P-wartości dla {column}: ", p_values)
        
        # Odchylenie
        std_dev = results.bse[column]
        
        # Obliczanie AIC, BIC, R-kwadrat Nagelkera
        AIC = results.aic
        BIC = results.bic
        R2_Nagelkera = results.prsquared

        # Test Hosmera-Lemeshowa 
        hl_stat, p_value = hosmer_lemeshow(y, results.predict(X))
        print('Hosmer-Lemeshow test: H=%.3f, p=%.3f' % (hl_stat, p_value))
        
        # Iloraz szans
        odds_ratio = np.exp(results.params[column])
        
        print(AIC, BIC, R2_Nagelkera, odds_ratio)
        new_data = pd.DataFrame({
            'Column' : [str(args)],
            'Std dev' : [std_dev],
            'AIC' : [AIC / 1000],
            'BIC' : [BIC / 1000],
            'R2_Nagelkera' : [R2_Nagelkera * 100],
            'Odds ratio' : [odds_ratio],
            'Hosmel test' : [hl_stat],
            'Significance': [p_values[1]],
        })    
        
        evaluation = pd.concat([evaluation, new_data], ignore_index=True)
        
        args.pop()
        
    
    return evaluation

In [219]:
res = evaluate()
res

Optimization terminated successfully.
         Current function value: 0.686949
         Iterations 4
P-wartości dla Płeć:  const    0.071860
Płeć     0.022628
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
587.9066530240073 596.010831361856 0.008906456482549618 1.615354174910962
Optimization terminated successfully.
         Current function value: 0.693013
         Iterations 3
P-wartości dla Stan cywilny:  const           0.959563
Stan cywilny    0.761094
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
593.0613644413224 601.1655427791712 0.00015711071308688318 1.1119155354449468
Optimization terminated successfully.
         Current function value: 0.683021
         Iterations 4
P-wartości dla Mieszkanie:  const         0.020086
Mieszkanie    0.003659
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
584.5679800436183 592.672158381467 0.01457335069867638 1.8551058620193004
Optimization terminated successfully.
         Current function value: 0.692955
         Iterations 3
P-wa

/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:11: RuntimeWarning: invalid value encountered in divide
  expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:11: RuntimeWarning: invalid value encountered in divide
  expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/f

,Column,Std dev,AIC,BIC,R2_Nagelkera,Odds ratio,Hosmel test,Significance
0,['Płeć'],0.210363,0.587907,0.596011,0.890646,1.615354,NaN,0.022628
1,['Stan cywilny'],0.348910,0.593061,0.601166,0.015711,1.111916,NaN,0.761094
2,['Mieszkanie'],0.212633,0.584568,0.592672,1.457335,1.855106,NaN,0.003659
3,['Praca'],0.228849,0.593012,0.601116,0.024093,0.917414,NaN,0.706432
4,['Okres'],0.008601,0.571708,0.579813,3.640046,0.962479,NaN,0.000009
5,['Wiek'],0.008910,0.590049,0.598154,0.526942,1.015720,21.953839,0.079999


In [220]:
res = evaluate('Okres')
res

Optimization terminated successfully.
         Current function value: 0.657460
         Iterations 5
P-wartości dla Płeć:  const    0.031029
Okres    0.000002
Płeć     0.003197
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
564.8407666590539 576.9970341658271 0.051452020932259646 1.9116346596484242
Optimization terminated successfully.
         Current function value: 0.667887
         Iterations 5
P-wartości dla Stan cywilny:  const           0.000065
Okres           0.000009
Stan cywilny    0.947275
dtype: float64
Hosmer-Lemeshow test: H=nan, p=nan
573.704079040439 585.8603465472122 0.03640788394585026 0.9765159430226884
Optimization terminated successfully.
         Current function value: 0.660965
         Iterations 5
P-wartości dla Mieszkanie:  const         0.081722
Okres         0.000030
Mieszkanie    0.015707
dtype: float64
Hosmer-Lemeshow test: H=7.029, p=0.533
567.8205872751287 579.9768547819019 0.046394224522273375 1.6942988963794474
Optimization terminated successfully

/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:11: RuntimeWarning: invalid value encountered in divide
  expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:10: RuntimeWarning: invalid value encountered in divide
  actual = np.bincount(bins, weights=y) / np.bincount(bins)
/var/folders/p1/dql2s7650jqg6q1h81txqczc0000gn/T/ipykernel_5226/3280109806.py:11: RuntimeWarning: invalid value encountered in divide
  expected = np.bincount(bins, weights=y_pred_prob) / np.bincount(bins)


,Column,Std dev,AIC,BIC,R2_Nagelkera,Odds ratio,Hosmel test,Significance
0,"['Okres', 'Płeć']",0.219784,0.564841,0.576997,5.145202,1.911635,NaN,0.000002
1,"['Okres', 'Stan cywilny']",0.359362,0.573704,0.585860,3.640788,0.976516,NaN,0.000009
2,"['Okres', 'Mieszkanie']",0.218272,0.567821,0.579977,4.639422,1.694299,7.029375,0.000030
3,"['Okres', 'Praca']",0.240826,0.573144,0.585300,3.735917,1.198282,14.920966,0.000007
4,"['Okres', 'Wiek']",0.009215,0.570486,0.582643,4.186969,1.016541,10.490896,0.000008


In [221]:
res = evaluate('Okres', 'Płeć')
res

Optimization terminated successfully.
         Current function value: 0.656789
         Iterations 5
P-wartości dla Stan cywilny:  const           0.026988
Okres           0.000002
Płeć            0.002388
Stan cywilny    0.449050
dtype: float64
Hosmer-Lemeshow test: H=8.200, p=0.414
566.2702775508683 582.478634226566 0.05242034021555875 0.7547437966070643
Optimization terminated successfully.
         Current function value: 0.652810
         Iterations 5
P-wartości dla Mieszkanie:  const         0.437027
Okres         0.000007
Płeć          0.008948
Mieszkanie    0.047270
dtype: float64
Hosmer-Lemeshow test: H=4.679, p=0.791
562.8887092222456 579.0970658979433 0.05816004282059706 1.5538420576080356
Optimization terminated successfully.
         Current function value: 0.656820
         Iterations 5
P-wartości dla Praca:  const    0.127509
Okres    0.000002
Płeć     0.003231
Praca    0.461084
dtype: float64
Hosmer-Lemeshow test: H=4.176, p=0.841
566.2969128635385 582.5052695392362 0.

,Column,Std dev,AIC,BIC,R2_Nagelkera,Odds ratio,Hosmel test,Significance
0,"['Okres', 'Płeć', 'Stan cywilny']",0.371700,0.566270,0.582479,5.242034,0.754744,8.200308,0.000002
1,"['Okres', 'Płeć', 'Mieszkanie']",0.222156,0.562889,0.579097,5.816004,1.553842,4.678788,0.000007
2,"['Okres', 'Płeć', 'Praca']",0.243805,0.566297,0.582505,5.237513,1.196858,4.176011,0.000002
3,"['Okres', 'Płeć', 'Wiek']",0.009372,0.565091,0.581300,5.442128,1.012421,6.978280,0.000002


In [222]:
res = evaluate('Okres', 'Płeć', 'Mieszkanie')
res

Optimization terminated successfully.
         Current function value: 0.651944
         Iterations 5
P-wartości dla Stan cywilny:  const           0.415412
Okres           0.000006
Płeć            0.006277
Mieszkanie      0.042899
Stan cywilny    0.389745
dtype: float64
Hosmer-Lemeshow test: H=3.196, p=0.921
564.152746078573 584.4131919231951 0.059409229340707914 0.7261668407771731
Optimization terminated successfully.
         Current function value: 0.652213
         Iterations 5
P-wartości dla Praca:  const         0.691573
Okres         0.000006
Płeć          0.008996
Mieszkanie    0.048278
Praca         0.476470
dtype: float64
Hosmer-Lemeshow test: H=4.118, p=0.846
564.3813010598449 584.6417469044669 0.0590212917052928 1.1909100301982625
Optimization terminated successfully.
         Current function value: 0.650715
         Iterations 5
P-wartości dla Wiek:  const         0.670911
Okres         0.000009
Płeć          0.018324
Mieszkanie    0.046393
Wiek          0.184039
dtype: 

,Column,Std dev,AIC,BIC,R2_Nagelkera,Odds ratio,Hosmel test,Significance
0,"['Okres', 'Płeć', 'Mieszkanie', 'Stan cywilny']",0.372030,0.564153,0.584413,5.940923,0.726167,3.195501,0.000006
1,"['Okres', 'Płeć', 'Mieszkanie', 'Praca']",0.245393,0.564381,0.584642,5.902129,1.190910,4.118397,0.000006
2,"['Okres', 'Płeć', 'Mieszkanie', 'Wiek']",0.009460,0.563108,0.583368,6.118325,1.012646,4.552182,0.000009
